<a href="https://colab.research.google.com/github/jsalbr/m3nlp/blob/main/Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering

Dieses Notebook sollte mit GPU ausgeführt werden.  
Dafür bitte zunächst im Menü "Laufzeit"->"Laufzeittyp ändern"->"Hardwarebeschleuniger: GPU" einstellen.


Credits: Das Notebook verwendet Ideen von
  * Natural Language Processing with Transformers von Lewis Tunstall, Leandro von Werra, Thomas Wolf, O'Reilly 2021, https://www.oreilly.com/library/view/natural-language-processing/9781098103231/
  * Heise Academy NLP-Kurs von Christian Winkler, https://github.com/heiseacademy/nlp-course/tree/main/09_Transfer_Learning
  * Haystack Tutorial von deepset.io, https://github.com/deepset-ai/haystack#mortar_board-tutorials

## System vorbereiten

### Installation von Transformers und Haystack

Achtung: In diesem Notebook werden sowohl die [Transformers-Bibliothek von HuggingFace](https://huggingface.co/transformers/) als auch [Haystack von deepset.ai](https://haystack.deepset.ai/) eingesetzt. 

Leider haben in der aktuellen Version beide Bibliotheken inkompatible Dependencies.
Die Installation hier funktioniert für diese Beispiele, aber es gibt eine Warnung am Ende. In der Praxis kann
es daher zu Problemen kommen. Für produktive Zwecke sollte deshalb mit getrennten
virtuellen Environments arbeiten.

**Geduld:** Die Installation dauert einen Moment.

In [ ]:
!pip install farm-haystack==0.10.0 grpcio==1.41.0
!pip install transformers==4.12.3 datasets
!pip install readability-lxml

In [ ]:
# patch for transformers 4.12.3, see https://github.com/huggingface/transformers/issues/14311
!wget https://raw.githubusercontent.com/jsalbr/m3nlp/main/question_answering.patch
!patch /usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py -i question_answering.patch

In [ ]:
!patch /usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py -i question_answering.patch

In [ ]:
%load_ext autoreload
%autoreload 2

### Noch ein paar Standard-Einstellungen setzen ...

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 200 # default 50; -1 = all
pd.options.display.float_format = '{:.2f}'.format

from textwrap import wrap, fill

In [ ]:
# suppress warnings
import warnings;
warnings.filterwarnings('ignore');

### Und eine kleine Anzeige-Funktion ...

welche mit Antworten sowohl von Transformer als auch von Haystack umgehen kann.

In [ ]:
from IPython.display import display, HTML

def display_qa(answers, question='', context='', padding=50):
    if type(answers) != list:
        answers = [answers]
    html = "<table>"
    if len(question) > 0: 
        html += f"<tr><td>Question:</td><td><span style='font-weight:bold'>{question}</span></td></tr>"
        html += f"<tr><td>&nbsp;<td><td> </td></tr>"
    for a in answers:
        if len(a['answer']) > 0:
            html += f"<tr><td>Answer:</td><td><span style='font-weight:bold'>{a['answer']}</span></td></tr>"
        else:
            html += f"<tr><td>Answer:</td><td>answer impossible</td></tr>"
        html += f"<tr><td>Score:</td><td>{a['score']}</td></tr>"
        start = a.get('start', a.get('offset_start'))
        end = a.get('end', a.get('offset_end'))
        html += f"<tr><td>Span:</td><td>{start}:{end}</td></tr>"
        ctx = a.get('context', context)
        if len(a['answer']) > 0 and len(ctx) > 0:
            left = max(0, start-padding)
            right = min(end+padding, len(ctx))
            html += "<tr><td>Snippet:</td><td>"
            html += f"{ctx[left:start]}<span style='color:blue;font-weight:bold'>"
            html += ctx[start:end]
            html += f"</span>{ctx[end:right]}</td>"
        html += f"<tr><td>&nbsp;<td><td> </td></tr>"
    html += '</table><br/>'
    display(HTML(html))


## Arbeit mit einem QA-Modell

Zunächst nutzen wir die [HuggingFace Transformers Library](https://huggingface.co/transformers/), um mit einem vortrainierten QA-Modell zu arbeiten.

### Modell laden

Eine Übersicht über die QA-Modelle auf dem HuggingFace Hub gibt's hier:  
https://huggingface.co/models?pipeline_tag=question-answering&sort=downloads

Wir nutzen dieses, weil es bei den Beispielen sehr gute Ergebnisse geliefert hat:  
https://huggingface.co/Sahajtomar/German-question-answer-Electra


In [ ]:
from transformers import pipeline

model_name = "Sahajtomar/German-question-answer-Electra"
# device = 0 is GPU
qa = pipeline("question-answering", model=model_name, tokenizer=model_name, device=0)

### Fragen zu Artikel beantworten

Zunächst das Grundprinzip: Das Modell beantwortet Fragen basierend auf dem Kontext, z.B. ein Wikipedia-Eintrag, ein News-Artikel oder ein User-Post.

Hier geht es um diesen Beispielartikel:  
https://www.heise.de/news/Giga-Factory-Berlin-fast-fertig-Erstes-Tesla-Model-Y-noch-dieses-Jahr-6213528.html

In [ ]:
context = """Giga Factory Berlin fast fertig – Erstes Tesla Model Y noch dieses Jahr

Elon Musk hat in knapp zwei Jahren eine riesige Fabrik vor die Tore Berlins gesetzt. 
Samstag ließ er erstmals Bürger ein. Nicht alle Nachbarn sind begeistert.

Der US-Elektroautobauer Tesla will spätestens im Dezember in Deutschland die Produktion 
für Europa starten. Dies kündigte Firmengründer Elon Musk am Wochenende bei einem Bürgerfest 
in seinem ersten europäischen Werk bei Berlin an. Kritik von Anwohnern und Umweltschützern 
an der in nur zwei Jahren konzipierten und errichteten Industrieanlage widersprach er. 
Ziel sei "eine wunderschöne Fabrik in Harmonie mit ihrer Umgebung".

Künftig sollen etwa 12.000 Mitarbeiter in Grünheide bis zu 500.000 Elektroautos im Jahr bauen. 
Dabei will Tesla möglichst viele Teile vor Ort produzieren, um von Zulieferern unabhängig zu sein. 
Tesla betont vor allem die Bedeutung der eigenen Druckgussanlage und der hochmodernen Lackiererei. 
Zudem entsteht neben dem Autowerk eine eigene Batteriefabrik.
"""

Jetzt können wir Fragen stellen:

In [ ]:
question="Wer ist Elon Musk?"
answer = qa(question, context)
answer['answer']

In [ ]:
question="Wer ist der Firmengründer?"
answer = qa(question, context)
answer['answer']

In [ ]:
question="Wer ist gründete Tesla?"
answer = qa(question=question, context=context)
answer['answer']

In [ ]:
question="Was ist Tesla?"
answer = qa(question=question, context=context)
answer['answer']

In [ ]:
question="Wer ist Tesla?"
answer = qa(question=question, context=context)
answer['answer']

In [ ]:
question="Wer ist begeistert?"
# question="Wer ist nicht begeistert?"
answer = qa(question=question, context=context)
display_qa(answer, question, context)

In [ ]:
question="Wer ist der Bundeskanzler?"
answer = qa(question=question, context=context) # handle_impossible_answer=True, top_k=3)
display_qa(answer, question, context)

In [ ]:
question="Wann ist Weihnachten?"
answer = qa(question=question, context=context, handle_impossible_answer=True) # False
answer['answer']
display_qa(answer, question, context)

In [ ]:
question="Wieviele Mitarbeiter?" 
answer = qa(question=question, context=context, handle_impossible_answer=True, top_k=5)
display_qa(answer, question, context)

In [ ]:
question="Was ist wichtig?"
answer = qa(question=question, context=context, handle_impossible_answer=True, top_k=5)
display_qa(answer, question, context)

### Fragen zu Wikipedia beantworten

Für einen längeren Text einen sich Wikipedia-Artikel, wie hier dieser zu "Game of Thrones":  
https://de.wikipedia.org/wiki/Game_of_Thrones

In [ ]:
from readability import Document
import requests
from bs4 import BeautifulSoup
doc = Document(requests.get("https://de.wikipedia.org/wiki/Game_of_Thrones", stream=True).text)
soup = BeautifulSoup(doc.summary())            
context = soup.text
len(context)

Das sind ca. 100kB!

In [ ]:
question="Wer sind die Geschwister von Arya?"
answer = qa(question=question, context=context)
display_qa(answer, question, context)

In [ ]:
question="Wen heiratet Tyrion?"
answer = qa(question=question, context=context)
display_qa(answer, question, context)

In [ ]:
question="Wer ist Tyrion Lennister?"
answer = qa(question=question, context=context) #, top_k=5)
display_qa(answer, question, context)

In [ ]:
question="Wann stirbt Eddard Stark?"
answer = qa(question=question, context=context, top_k=5, handle_impossible_answers=True)
display_qa(answer, question, context)

In [ ]:
question="Wo stirbt Eddard Stark?"
answer = qa(question=question, context=context, top_k=5, handle_impossible_answers=True)
display_qa(answer, question, context)

## Deep Dive

In [ ]:
question = "Wie viele Menschen leben in Berlin?"
context = "In Deutschland leben ca. 80 Millionen Menschen, allein in Berlin ca. 4 Mio."

answer = qa(question=question, context=context)
display_qa(answer, question, context, padding=1000)

In [ ]:
model_name = "Sahajtomar/German-question-answer-Electra"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer(question, context, return_tensors="pt");
inputs

In [ ]:
qa_df = pd.DataFrame(
    [tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]),
     inputs['input_ids'][0].numpy(),
     inputs['token_type_ids'][0].numpy(),
     inputs['attention_mask'][0].numpy()]).T

qa_df.columns=['token', 'id', 'type', 'attn']
qa_df.T

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
outputs = model(**inputs)

In [ ]:
def maxval_in_col(column):    
    highlight = 'background-color: palegreen;'
    return [highlight if v == column.max() else '' for v in column]

qa_df = pd.concat([qa_df, 
                   pd.Series(outputs['start_logits'][0].detach(), name='start'),
                   pd.Series(outputs['end_logits'][0].detach(), name='end')], axis=1)

# answer span must be in context (type==1)
qa_df.query('type==1').style.apply(maxval_in_col, subset=['start', 'end'], axis=0)

In [ ]:
import torch

start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]
answer = tokenizer.decode(answer_span)
print(f"Question: {question}")
print(f"Answer:   {answer}")

## Retriever-Reader mit Haystack

Nun wird ein größeres Szenario simuliert. Stellen Sie sich vor, Sie haben sehr viele Dokumente und suchen darin Antworten. Da suchen Sie die Nadel im Heuhaufen - ein Fall für [Haystack](https://haystack.deepset.ai/).


### Anwendungsbeispiel: Aspect-based Sentiment Analysis

An dieser Stelle soll ein praktisches Anwendungsbeispiel gezeigt werden.
Es geht darum, Kunden-Meinungen zu bestimmten Eigenschaften eines Produkts herauszufinden. Dafür werden Amazon-Reviews zu dem Produkt mit einem QA-Modell "befragt". 

Da wir hier nicht nur einen (Kon-)Text auszuwerten haben, sondern viele Rewiews, wird ein Retriever-Reader-Modell benutzt. Dabei werden durch den Retriever die relevanten Kommentare vorselektiert, um dann durch den Reader ausgewertet zu werden.

Der Datensatz, den wir verwenden, ist ein Auszug aus dem "Subjective QA" Datensatz, den man direkt vom HuggingFace Hub beziehen kann:  
https://huggingface.co/datasets/subjqa

In [ ]:
from datasets import load_dataset

# other options include: books, grocery, movies, restaurants, tripadvisor
subjqa = load_dataset("subjqa", "electronics")
subjqa.set_format("pandas")

# flatten the nested dataset columns for easy access
df = [ds[:] for split, ds in subjqa.flatten().items() if split == 'train'][0]

# select some columns
df = df[["title", "question", "answers.text", "answers.answer_start", "context"]]
df = df.drop_duplicates(subset="context").rename(columns={"answers.text": "answer", "answers.answer_start": "start"})

print(list(df.columns))
print(f"\n{len(df)} rows")

Schauen wir uns ein paar Datensätze an:

In [ ]:
df.sample(3)

Auswertung nach Fragetypen:

In [ ]:
counts = {}
question_types = ["What", "How", "Is", "Does", "Do", "Was", "Where", "Why"]

for q in question_types:
    counts[q] = df["question"].str.startswith(q).value_counts()[True]

pd.Series(counts).sort_values().plot(kind="barh");

### Befüllen des Document Stores für den Retriever



Haystack unterstützt folgende Document Stores:
  * Elasticsearch (Sparse BM25/TF-IDF + Dense Vectors, https://elastic.co)
  * FAISS (von Facebook AI für Dense Vectors, https://faiss.ai/)
  * SQL (SQLite, PostgreSQL, MySQL)
  * InMemoryDocumentStore

Der Einfachheit halber wird hier der InMemoryDocumentStore genutzt. Für die Praxis wird aber ElasticSearch empfohlen, weil dieser Such-Index neben einer Volltextsuche eine Vielzahl von Filtermöglichkeiten für Metadaten bietet.


Ein Document-Store erwartet folgendes Input-Format:
```python
docs = [
    {
        'text': DOCUMENT_TEXT_HERE,
        'meta': {'name': DOCUMENT_NAME, 'category': DOCUMENT_CATEGORY}
    }, ...
]
```


Für den `InMemoryDocumentStore` wird an dieser Stelle schon einmal auf den zu analysierenden Artikel gefiltert. Wir nehmen diesen hier:

**Panasonic ErgoFit In-Ear Earbud Headphones RP-HJE120-D (Orange) Dynamic Crystal Clear Sound, Ergonomic Comfort-Fit**  
<img src="https://m.media-amazon.com/images/I/31oE5NluLhL.jpg" width="100"/>

https://www.amazon.com/dp/B003ELYQGG  
https://amazon-asin.com/asincheck/?product_id=B003ELYQGG


In [ ]:
# create docs (in the example for one item only)
item_id = "B003ELYQGG"

docs = []
for _, row in df.query(f"title == '{item_id}'").iterrows():
    doc = {"text": row["context"],
           "meta": {"item_id": row["title"]}}
    docs.append(doc)

docs[:3]

In [ ]:
from haystack.document_store import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

document_store.write_documents(docs, index="document")
print(f"{document_store.get_document_count()} docs loaded.")

### Dokumenten-Suche mit dem Retriever

In [ ]:
from haystack.retriever.sparse import TfidfRetriever
retriever = TfidfRetriever(document_store=document_store)

question = "How is the bass?"
retrieved_docs = retriever.retrieve(query=question, top_k=3)
# ElasticSearch would support real filters
# retrieved_docs = retriever.retrieve(query=question, top_k=3, filters={"item_id":[item_id]]})

for doc in retrieved_docs:
    print(fill(doc.text), end="\n\n")

### Antworten bekommen mit dem Reader

Haystack unterstützt zwei Reader, den `FARMReader` und den `TransformersReader`. Beide nutzen Transformer-Modelle, unterscheiden sich aber in kleinen Details, die [hier](https://haystack.deepset.ai/docs/latest/readermd#deeper-dive-farm-vs-transformers) erläutert werden.

In [ ]:
from haystack.reader.farm import FARMReader

reader = FARMReader(model_name_or_path=model_name, progress_bar=False,
                    max_seq_len=256, doc_stride=128, # these are defaults
                    return_no_answer=False, use_gpu=True)

In [ ]:
question = "How is the bass?"
answers = reader.predict_on_texts(question=question, texts=[retrieved_docs[1].text], top_k=3)
answers

Haben Sie bemerkt, dass wir immer noch das gleiche Modell verwendet haben mit dem wir auch schon die deutschen Texte analysiert haben?

Das ist mit einem multilingualen Modell möglich! Mit einem rein englischen Modell werden aber auf englischen Texten bessere Ergebnisse erreicht.

### Retriever und Reader in der Haystack-Pipeline

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
question = "How is the bass?"
# question = "Do they sound good?"
# question = "How do they fit?"
answers = pipe.run(query=question, params={"Retriever": {"top_k": 10}, 
                                         "Reader": {"top_k": 5}})

display_qa(answers['answers'], question, padding=500)

### Und natürlich eine WordCloud zum Abschluss 😀

In diesem Beispiel wird aus allen Dokumenten (wir haben nur 35) jeweils die Meinung zum Bass erfragt. Die eindeutigen Antworten werden gezählt und mit einer WordCloud visualisiert. Bei sehr vielen Reviews kann man sich so sehr schnell ein Meinungsbild verschaffen.

In [ ]:
from collections import Counter

question = "How is the bass?"
retrieved_docs = retriever.retrieve(query=question, top_k=100)

counter = Counter()
for doc in retrieved_docs:
    answer = reader.predict_on_texts(question=question, texts=[doc.text], top_k=1)['answers'][0]['answer']
    if len(answer) < 30:
        counter.update([answer])

counter

In [ ]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

wc = WordCloud(width=800, height=400, background_color= "black", colormap="Paired")
wc.generate_from_frequencies(counter)
plt.figure(figsize=(16, 8))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")